# Neural Networks and League of Legends

In this notebook, we take a look at League of Legends datasets and how to use them together with Auto Machine Learning (AutoML) tools. Source code and additional resources are available at https://github.com/oracle-devrel/leagueoflegends-optimizer/tree/livelabs


This notebook creates several AutoML fine-tuned models, to make predictions based on pre-existing data (extracted using _`leagueoflegends-optimizer)`_.

## 1. Offline DataSet

Firstly, we'll explore the offline dataset that can be inferred from the _`https://REGION.api.riotgames.com/lol/match/v5/matches/ID`_ endpoint
    
Downloading data from professional games, we built the dataset contained within _`matchups.json`_. 
    
For each match, we have 5 matchups (5 players play against each other in different roles and different places in the map), just like this one:
    
![](https://github.com/oracle-devrel/leagueoflegends-optimizer/raw/livelabs/hols/nn/model_building/images/matchups.png)

In [ ]:
from pandas_profiling import ProfileReport
from autogluon.tabular import TabularPredictor, TabularDataset
import pandas as pd
import json

In [ ]:
# Read the dataset from local file
df = pd.read_json('/Users/mattgunnin/Sites/AI/20_GPT/gpt_lolcoach/data/lol/export.json', orient='records')

In [ ]:
# HTML report for Exploratory Data Analysis
report = ProfileReport(df, title="Matchups Exploration", html={'style': {'full_width': True}})

report.to_notebook_iframe()

# more information about ProfileReport can be found here: https://pypi.org/project/pandas-profiling/

# profile.to_file("your_report.html")

In [ ]:
# Display 5 first rows
df.head(5)

In [ ]:
# Display list of columns in the dataset
df.columns

In [ ]:
# Get some stats from pandas (also findable in the AutoGluon HTML report)
df.describe()

In [ ]:
# Display the first row
df.iloc[0]

In [ ]:
# drop columns we don't want (identifiers)
# We remove these variables as we can't infer anything useful from ID columns in this case
def _drop_column(df, col_name=list()):
    for x in col_name:
        try:
            df.drop([x],
                axis=1,
                inplace=True) # inplace=True changes the value by reference.
        except KeyError:
            print('{} already dropped from df'.format(x))
        

    return df



df = _drop_column(df,
    col_name=['summonername', 'p_match_id','puuid'])

In [ ]:
# Perform 80-20% train-test split
train = df.sample(frac=0.8,random_state=200) #random state is a seed value
test = df.drop(train.index)

In [ ]:
# New look at the data
df.iloc[0]

In [ ]:
# Create AutoGluon Tabular Dataset 
# https://auto.gluon.ai/stable/tutorials/tabular_prediction/index.html
df = TabularDataset(df)

In [ ]:
# Determine the label in our TabularDataset we want to predict
label = 'win'


# Create the tabular predictor with the target label by passing the training test
predictor = TabularPredictor(label=label,
                path='./autogluon_trained_models_liveclient_classifier').fit(train, time_limit=60)

In [ ]:
# We load the model 
predictor = TabularPredictor.load('./autogluon_trained_models_liveclient_classifier')

In [ ]:
# Make a test prediction from original test dataset
y_test = test[label] # we want 'win' column to be predicted


test_data_nolabel = test.drop(columns=[label])  # delete label column, also drop identifier column
# We have the testing dataset ready
test_data_nolabel.head(5)

In [ ]:
# Predict test values
y_pred = predictor.predict(test_data_nolabel)
print("Predictions:  \n", y_pred)

# Evaluate prediction performance
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

In [ ]:
# Best performing models
predictor.leaderboard(test, silent=False)

In [ ]:
# For the best model, display feature importance
predictor.feature_importance(test,
                            subsample_size=5000)

In [ ]:
# Making predictions from incoming data
# goldearned, totalminionskilled, win, kills, assists, deaths, champion, visionscore, totaldamagedealttochampions, gameversion
# [5506, 134, false, 0, 1, 3, Jayce, 7, 2350, 11.15.389.2308]
data = [5506, 134, 0, 1, 3, 'Jayce', 7, 2350, '11.15.389.2308']

# From a list, load it into a dataframe and specify column names for consistency
test_d = pd.DataFrame([data], columns=['goldearned', 'totalminionskilled', 'kills', 'assists', 'deaths',
                                       'champion', 'visionscore', 'totaldamagedealttochampions', 'gameversion'])

In [ ]:
#data = [91, 35, 0, 1080321, 2204, 390, 225, 0, 10, 0, 672, 0, 220, 0, 94, 0, 1047, 33]
#test_d = pd.DataFrame([data], columns=['magicResist', 'healthRegenRate', 'spellVamp', 'timestamp', 'maxHealth', 'moveSpeed', 'attackDamage', 'armorPenetrationPercent', 'lifesteal', 'abilityPower', 'resourceValue', 'magicPenetrationFlat', 'attackSpeed', 'currentHealth', 'armor', 'magicPenetrationPercent', 'resourceMax', 'resourceRegenRate'])

In [ ]:
# Make prediction
predictor.predict(test_d)

# Print how probable each class is
print(predictor.predict_proba(test_d).iloc[0])

## Exploring Live Client DataSet + Data Points

In this second part of the notebook, we're going to focus on data that can be obtained from League of Legends **in real time**. Our data structure will be different, but it will help us create a "live companion" integration to help us while we're playing the game.

This dataset (_`live_client_dataset.csv`_) uses [this function](https://github.com/oracle-devrel/leagueoflegends-optimizer/blob/livelabs/src/league.py#L487) to download previous professional matches' timelines, with all kinds of events, and extracts. The Riot Games API endpoint used is _`https://MATCHID.api.riotgames.com/lol/match/v5/matches/REGION/timeline`_.
 
![](https://github.com/oracle-devrel/leagueoflegends-optimizer/blob/livelabs/hols/nn/model_building/images/liveclient.png?raw=True)

In [ ]:
# Imports

from pandas_profiling import ProfileReport
from autogluon.tabular import TabularPredictor, TabularDataset
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json

In [ ]:
# Read the dataset from local file
df = pd.read_csv('/home/datascience/datasets/matchups_livelabs.csv', engine='python')

In [ ]:
# HTML report for Exploratory Data Analysis
report = ProfileReport(df, title="Matchups (LiveLabs Format) Exploration", html={'style': {'full_width': True}})

report.to_notebook_iframe()

In [ ]:
# Display 5 first rows
df.head(5)

In [ ]:
# Display list of columns in the dataset
df.columns

In [ ]:
# Get some stats from pandas (also findable in the AutoGluon HTML report)
df.describe()

In [ ]:
df.iloc[0]

In [ ]:
"""
BONUSARMORPENETRATIONPERCENT has constant value "0" 	Constant
BONUSMAGICPENETRATIONPERCENT has constant value "0" 	Constant
COOLDOWNREDUCTION has constant value "0" 	Constant
ARMORPENETRATIONFLAT has constant value "0" 	Constant
IDENTIFIER has a high cardinality: 34990 distinct values 
"""


# drop columns we don't want (identifiers + constant columns)
df = _drop_column(df, col_name=['BONUSARMORPENETRATIONPERCENT', 'BONUSMAGICPENETRATIONPERCENT', 'COOLDOWNREDUCTION',
    'ARMORPENETRATIONFLAT', 'IDENTIFIER', 'Unnamed: 24']) # Unnamed: 24 is a null column, just remove it

In [ ]:
# Perform 80-20% train-test split
train = df.sample(frac=0.8,random_state=200)
test = df.drop(train.index)

In [ ]:
# New look at the data
df.iloc[0]

In [ ]:
# Create AutoGluon Tabular Dataset 
# https://auto.gluon.ai/stable/tutorials/tabular_prediction/index.html
df = TabularDataset(df)

In [ ]:
# Determine the label in our TabularDataset we want to predict
label = 'WINNER'


# Create the tabular predictor with the target label by passing the training test
predictor = TabularPredictor(label=label,
                path='./ag_hol2_livelabs').fit(train, time_limit=120)

In [ ]:
# We load the model 
predictor = TabularPredictor.load('./ag_hol2_livelabs')

In [ ]:
# Best performing models
#predictor.leaderboard(test, silent=False) # this takes a while...

In [ ]:
# For the best model, display feature importance
#predictor.feature_importance(test,
#                            subsample_size=5000) # this also takes a while depending on the subsample size

In [ ]:
# Making predictions from incoming data
# goldearned, totalminionskilled, win, kills, assists, deaths, champion, visionscore, totaldamagedealttochampions, gameversion
'''MAGICRESIST                    29
HEALTHREGENRATE                17
SPELLVAMP                       0
TIMESTAMP                  180033
MAXHEALTH                     714
MOVESPEED                     345
ATTACKDAMAGE                   65
ARMORPENETRATIONPERCENT         0
LIFESTEAL                       0
ABILITYPOWER                   15
RESOURCEVALUE                  70
MAGICPENETRATIONFLAT            0
ATTACKSPEED                   102
CURRENTHEALTH                 513
ARMOR                          42
MAGICPENETRATIONPERCENT         0
RESOURCEMAX                   100
RESOURCEREGENRATE               0
'''
data = [29, 17, 0, 180033, 714, 345, 65, 0, 0, 15, 70, 0, 102, 513, 42, 0, 100, 0]

# From a list, load it into a dataframe and specify column names for consistency
test_d = pd.DataFrame([data], columns=['MAGICRESIST', 'HEALTHREGENRATE', 'SPELLVAMP', 'TIMESTAMP', 'MAXHEALTH',
                                       'MOVESPEED', 'ATTACKDAMAGE', 'ARMORPENETRATIONPERCENT', 'LIFESTEAL',
                                       'ABILITYPOWER', 'RESOURCEVALUE', 'MAGICPENETRATIONFLAT', 'ATTACKSPEED',
                                       'CURRENTHEALTH', 'ARMOR', 'MAGICPENETRATIONPERCENT', 'RESOURCEMAX',
                                       'RESOURCEREGENRATE'])

In [ ]:
print(test_d[:1])

In [ ]:
# Make prediction
predictor.predict(test_d)

# Print how probable each class is
print(predictor.predict_proba(test_d).iloc[0])